In [1]:
import pandas as pd 
import numpy as np
df = pd.read_excel('Выгрузка 2021.xlsx')
dfstart = pd.read_excel('Выгрузка 2021.xlsx')
df.head()

,Order Number,Subscription Start Date,renewal order,Customer Total Orders,Order Status,Order Date,First Name (Billing),Last Name (Billing),Email (Billing),Phone (Billing),...,Subscriber email,Subscription Last order date,Subscription On hold date,Subscription Cancelleted date,Subscription End date,Subscription Coupon,Subscription Total refund,Subscription Total revenue,Start Month,End Month
0,128186,2021-06-01 03:09:20,122879.0,5,Processing,2021-09-30 20:12:09,Debra,Passarella,debra.passarella@gmail.com,NaN,...,debra.passarella@gmail.com,2021-09-30 20:12:09,NaN,NaN,NaN,NaN,0.0,119.85,06-2021,NaN
1,128184,2021-10-01 01:26:35,NaN,1,Processing,2021-09-30 18:26:35,Brooks,Johnson,victoria.johnson@outlook.com,NaN,...,victoria.johnson@outlook.com,2021-09-30 18:26:35,NaN,NaN,NaN,NaN,0.0,0.00,10-2021,NaN
2,128183,2021-08-01 00:40:54,126373.0,3,Processing,2021-09-30 17:42:09,John,Cozzi,bigdawgcoz10@gmail.com,2095096317,...,bigdawgcoz10@gmail.com,2021-09-30 17:42:09,NaN,NaN,NaN,NaN,0.0,43.25,08-2021,NaN
3,128182,2021-07-31 23:55:01,126368.0,3,Processing,2021-09-30 16:55:03,Tiyi,Smith,tiyi56@yahoo.com,NaN,...,tiyi56@yahoo.com,2021-09-30 16:55:03,NaN,NaN,NaN,NaN,0.0,39.95,07-2021,NaN
4,128180,2021-09-30 22:27:08,NaN,1,Processing,2021-09-30 15:27:08,Barbara,Delcambre,barbara.delcambre@yahoo.com,NaN,...,barbara.delcambre@yahoo.com,2021-09-30 15:27:08,NaN,NaN,NaN,NaN,0.0,0.00,09-2021,NaN


In [2]:
def update_sub_start_date(df):
    print('Добавляем значения в поле Subscription Start Date для Bundle и Gift.')
    a = list(df['Subscription Start Date'])
    b = list(df['Order Date'])

    for i in range(len(a)):
        if str(a[i]) == 'nan':
            a[i] = str(b[i])
    df['Subscription Start Date'] = a
    return df

def update_full_names_for_catefories(df):
    print('Переименовываем названия наборов.')
    a = list(df['Full names for categories'])
    for i in range(len(a)):
        try:
            rez = ''
            if '3+' in a[i]:
                rez += '3+'
            if '5+' in a[i]:
                rez += '5+'
            if 'C&' in a[i] or '8+' in a[i]:
                rez += '8+'
            if len(rez) > 0:
                a[i] = rez
        except:
            continue
    df['Full names for categories'] = a
    return df

def del_test_coupons(df):
    print('Удаляем тестовые купоны.')
    coupons = list(df['Coupon Code'])
    for i in range(len(coupons)):
        try:
            if 'test' in coupons[i] or 'altdev' in coupons[i] or 'Altdev' in coupons[i]:
                coupons[i] = 'test'
        except:
            continue
    df['Coupon Code'] = coupons
    df.drop(df[df['Coupon Code'].isin(['test'])].index,inplace = True)
    return df             

def del_wrong_mails(df):
    print('Удалем некорректные почты.')
    d = ['F0e0n0i0k0s@gmail.com', 'masha20795@gmail.com','test@test.by',
         'mkrasnoschekova@gmail.com','anton.rogozhkin@gmail.com','plokhotnov@gmail.com','Ksenia_levenkova@yahoo.com']
    mails = list(df['Email (Billing)'])
    for i in range(len(mails)):
        if '@oysterkit.com' in mails[i] or '.ru' in mails[i]:
            mails[i] = 'test'
    df['Email (Billing)'] = mails
    df.drop(df[df['Email (Billing)'].isin(['test'])].index,inplace = True)
    return df

def add_sub_month(df):
    print('Добавляем столбцы sub_month к таблице.')
    a = list(df['Subscription Start Date'])
    for i in range(len(a)):
        try:
            a[i] = a[i].split()[0][:-3]
        except:
            a[i] = 'NaN'      
    df['sub_month'] = a
    return df

def rename_Item_name(df,file):
    print('Приводим к одному названию поле Item Name.')
    f = open(file)
    d = {}
    line = f.readline()
    while line:
        x = line.split('//')
        d[x[0].strip()] = x[1].strip()
        line = f.readline()
    names = list(df['Item Name'])
    for i in range(len(names)):
        try:
            names[i] = d[names[i].strip()]
        except:
            names[i] = 'test'
    df['Item Name'] = names
    return df

def add_year_and_month(df):
    print('Добавляем столбец Year и Month.')
    a = df['Order Date']
    year = []
    month = []
    for x in a:
        r = str(x).split()[0].split('-')
        year.append(r[0])
        month.append(int(r[1]))
    df['Year'] = year
    df['Month'] = month
    
    return df

def del_rus_names(df):
    print('Удаляем русифицированные имена и фамилии.')
    l = []
    for x in range(1040,1104):
        l.append(chr(x))

    fname = list(df['First Name (Billing)'])
    lname = list(df['Last Name (Billing)'])

    for i in range(len(fname)):
        try:
            for x in l:
                if x in fname[i]:
                    fname[i] = 'test'
        except:
            continue
    for i in range(len(lname)):
        try:
            for x in l:
                if x in lname[i]:
                    lname[i] = 'test'
        except:
            continue
    df['First Name (Billing)'] = fname
    df['Last Name (Billing)'] = lname

    df.drop(df[df['First Name (Billing)'].isin(['test'])].index,inplace = True)
    df.drop(df[df['Last Name (Billing)'].isin(['test'])].index,inplace = True)
    return df

def del_test_orders(df):
    print('Удаляем тестовые заказы.')
    bad_names = ['test','AltestTest','test 1','test 2','Test',
                 'alttest','Alttest','Altestat','test2','Test1','Test2','Ulugbek','ulugbek','Zayniev','zayniev']
    df.drop(df[df['Item Name'].isin(['test'])].index,inplace = True)
    df.drop(df[df['Last Name (Billing)'].isin(bad_names)].index,inplace = True)
    df.drop(df[df['First Name (Billing)'].isin(bad_names)].index,inplace = True)
    df.drop(df[df['Order Status'].isin(['Failed'])].index,inplace = True)
    return df        

In [3]:
df = update_sub_start_date(df)
df = del_wrong_mails(df)
df = del_test_coupons(df)
df = rename_Item_name(df,'lines.txt')
df = update_full_names_for_catefories(df)
df = del_test_orders(df)
df = del_rus_names(df)
df = add_year_and_month(df)
df = add_sub_month(df)

Добавляем значения в поле Subscription Start Date для Bundle и Gift.
Удалем некорректные почты.
Удаляем тестовые купоны.
Приводим к одному названию поле Item Name.
Переименовываем названия наборов.
Удаляем тестовые заказы.
Удаляем русифицированные имена и фамилии.
Добавляем столбец Year и Month.
Добавляем столбцы sub_month к таблице.


In [4]:
df3 = df[df['Full names for categories'].isin(['3+','3+5+','3+5+8+'])]
df5 = df[df['Full names for categories'].isin(['5+','3+5+','3+5+8+'])]
df8 = df[df['Full names for categories'].isin(['8+','3+5+','3+5+8+'])]

pivot_quantity3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount3 = pd.pivot_table(df3,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

pivot_quantity5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount5 = pd.pivot_table(df5,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

pivot_quantity8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Quantity'],aggfunc=len)
pivot_order_total_amount8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Total Amount'],aggfunc = np.sum)
pivot_order_refund_amount8 = pd.pivot_table(df8,index = ['sub_month','Item Name'],columns = ['Year','Month'],values = ['Order Refund Amount'],aggfunc = np.sum)

In [5]:
with pd.ExcelWriter('cohort analysis new v2.xlsx') as writer:  
    dfstart.to_excel(writer, sheet_name = 'сырая выгрузка (2021)')
    df.to_excel(writer, sheet_name = 'обработанная таблица')
    
    pivot_quantity3.to_excel(writer, sheet_name = 'quantity 3+')
    pivot_order_total_amount3.to_excel(writer, sheet_name = 'total_amount 3+')
    pivot_order_refund_amount3.to_excel(writer, sheet_name = 'total_refund 3+')
    
    pivot_quantity5.to_excel(writer, sheet_name = 'quantity 5+')
    pivot_order_total_amount5.to_excel(writer, sheet_name = 'total_amount 5+')
    pivot_order_refund_amount5.to_excel(writer, sheet_name = 'total_refund 5+')
    
    pivot_quantity8.to_excel(writer, sheet_name = 'quantity 8+')
    pivot_order_total_amount8.to_excel(writer, sheet_name = 'total_amount 8+')
    pivot_order_refund_amount8.to_excel(writer, sheet_name = 'total_refund 8+')